# Proof of concept
## Get PES of AL2 using Active Space and FreezeCore evaluation
- taken from: https://github.com/Qiskit/textbook/blob/main/notebooks/ch-applications/vqe-molecules.ipynb

In [1]:
# pylint: disable=line-too-long
import qiskit_nature
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

qiskit_nature.settings.use_pauli_sum_op = False  # pylint: disable=undefined-variable
# pylint: enable=line-too-long
from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2

import numpy as np
# SciPy minimizer routine
from scipy.optimize import minimize

from qiskit_algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator as AerEstimator

## Normal with Jordan-Wigner and TwoQubitReduction

In [2]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Al", "Al"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    problem = driver.run()

    fermionic_op = problem.hamiltonian.second_q_op()
    
    #mapper_jw = JordanWignerMapper()
    #qubit_jw_op = mapper_jw.map(fermionic_op)
    
    mapper_par = ParityMapper(num_particles=problem.num_particles)
    qubit_op = mapper_par.map(fermionic_op)
    print (f"Problem spatial orbitals {problem.num_spatial_orbitals}")
    print (f"Problem particles {problem.num_particles}")
    return qubit_op, problem.num_particles, problem.num_spatial_orbitals, problem, mapper_par

In [3]:
(qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op(1.7)

Problem spatial orbitals 18
Problem particles (13, 13)


## Applying FreezeCore

In [4]:
def get_qubit_op_fc(dist):
    # Define Molecule
    molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Al", "Al"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    problem = driver.run()

    # Now you can get the reduced electronic structure problem
    from qiskit_nature.second_q.transformers import FreezeCoreTransformer

    fc_transformer = FreezeCoreTransformer()

    fc_problem = fc_transformer.transform(problem)

    mapper = ParityMapper(num_particles=fc_problem.num_particles)
    qubit_op = mapper.map(fc_problem.second_q_ops()[0])
    print (f"Problem spatial orbitals {problem.num_spatial_orbitals}")
    print (f"Problem particles {problem.num_particles}")
    print (f"FC-Problem spatial orbitals {fc_problem.num_spatial_orbitals}")
    print (f"FC-Problem particles {fc_problem.num_particles}")
    return qubit_op, fc_problem.num_particles, fc_problem.num_spatial_orbitals, fc_problem, mapper

In [5]:
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

In [6]:
distances = np.arange(1.5, 3.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=100)
noiseless_estimator = AerEstimator(approximation=True)

# pylint: disable=undefined-loop-variable,line-too-long
for dist in distances:
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op_fc(
        dist
    )

    #result = exact_solver(qubit_op, problem)
    #exact_energies.append(result.total_energies[0].real)
    init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    var_form = UCCSD(
        num_spatial_orbitals, num_particles, mapper, initial_state=init_state
    )
    print(f"Qubits needed in ansatz: {var_form.num_qubits}")
    vqe = VQE(
        noiseless_estimator,
        var_form,
        optimizer,
        initial_point=[0] * var_form.num_parameters,
    )
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(
        f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Exact Energy: {exact_energies[-1]:.5f}",
    )

print("All energies have been calculated")

Problem spatial orbitals 18
Problem particles (13, 13)
FC-Problem spatial orbitals 8
FC-Problem particles (3, 3)
Qubits needed in ansatz: 14


KeyboardInterrupt: 

In [ ]:
plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_energies, label="VQE Energy")
plt.xlabel("Atomic distance (Angstrom)")
plt.ylabel("Energy")
plt.legend()
plt.show()

## Apply ActiveSpaceTransformer additionally

In [8]:
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

In [9]:
def get_qubit_op_as(dist):
    # Define Molecule
    molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Al", "Al"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    problem = driver.run()

    # Now you can get the reduced electronic structure problem
    from qiskit_nature.second_q.transformers import FreezeCoreTransformer

    fc_transformer = FreezeCoreTransformer()

    fc_problem = fc_transformer.transform(problem)

    # active space transformer
    from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

    as_transformer = ActiveSpaceTransformer(2, 2)

    as_problem = as_transformer.transform(fc_problem)
    
    num_particles = as_problem.num_particles
    num_spatial_orbitals = as_problem.num_spatial_orbitals

    mapper = ParityMapper(num_particles=num_particles)
    qubit_op = mapper.map(as_problem.second_q_ops()[0])
    print (f"Problem spatial orbitals {problem.num_spatial_orbitals}")
    print (f"Problem particles {problem.num_particles}")
    print (f"FC-Problem spatial orbitals {fc_problem.num_spatial_orbitals}")
    print (f"FC-Problem particles {fc_problem.num_particles}")
    print (f"AS-Problem spatial orbitals {as_problem.num_spatial_orbitals}")
    print (f"AS-Problem particles {as_problem.num_particles}")
    return qubit_op, as_problem.num_particles, as_problem.num_spatial_orbitals, as_problem, mapper

In [ ]:
distances = np.arange(1.5, 3.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=100)
noiseless_estimator = AerEstimator(approximation=True)

# pylint: disable=undefined-loop-variable,line-too-long
for dist in distances:
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op_as(
        dist
    )

    result = exact_solver(qubit_op, problem)
    exact_energies.append(result.total_energies[0].real)
    init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    var_form = UCCSD(
        num_spatial_orbitals, num_particles, mapper, initial_state=init_state
    )
    print(f"Qubits needed in ansatz: {var_form.num_qubits}")
    vqe = VQE(
        noiseless_estimator,
        var_form,
        optimizer,
        initial_point=[0] * var_form.num_parameters,
    )
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(
        f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Exact Energy: {exact_energies[-1]:.5f}",
    )

print("All energies have been calculated")

Problem spatial orbitals 18
Problem particles (13, 13)
FC-Problem spatial orbitals 8
FC-Problem particles (3, 3)
AS-Problem spatial orbitals 2
AS-Problem particles (1, 1)
Qubits needed in ansatz: 2
Interatomic Distance: 1.5 VQE Result: -477.44904 Exact Energy: -477.48401
Problem spatial orbitals 18
Problem particles (13, 13)
FC-Problem spatial orbitals 8
FC-Problem particles (3, 3)
AS-Problem spatial orbitals 2
AS-Problem particles (1, 1)
Qubits needed in ansatz: 2
Interatomic Distance: 1.6 VQE Result: -477.54879 Exact Energy: -477.58234
Problem spatial orbitals 18
Problem particles (13, 13)
FC-Problem spatial orbitals 8
FC-Problem particles (3, 3)
AS-Problem spatial orbitals 2
AS-Problem particles (1, 1)
Qubits needed in ansatz: 2
Interatomic Distance: 1.7 VQE Result: -477.61970 Exact Energy: -477.65193
Problem spatial orbitals 18
Problem particles (13, 13)
FC-Problem spatial orbitals 8
FC-Problem particles (3, 3)
AS-Problem spatial orbitals 2
AS-Problem particles (1, 1)
Qubits needed

In [ ]:
plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_energies, label="VQE Energy")
plt.xlabel("Atomic distance (Angstrom)")
plt.ylabel("Energy")
plt.legend()
plt.show()